In [5]:
import requests
import csv
import time
import base64
import json


In [6]:
def extract_users(endpoint, headers):
    response = requests.get(endpoint, headers=headers)
    data = response.json()

    # Extract users' email and id and append them to a CSV file
    with open('zendesk_users.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for user in data['users']:
            email = user['email']
            user_id = user['id']
            writer.writerow([email, user_id])

    # Check if there is a next page
    if data['next_page']:
        time.sleep(0.2)
        next_page = data['next_page']
        print("Processing page:", next_page)
        extract_users(data['next_page'], headers)

In [7]:
with open('config.json') as config_file:
    config = json.load(config_file)

username = config['username'] + "/token"
password = config['api_token']
hostname = config['zndsk_subdomain']

In [8]:
headers = {
    'Authorization': 'Basic ' + base64.b64encode(f'{username}:{password}'.encode()).decode()
}

In [9]:
# Start extraction from the first page
endpoint = f'https://{hostname}.zendesk.com/api/v2/users.json?page=1&role=end-user'
extract_users(endpoint, headers)